## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\rverd\OneDrive\Bureaublad\Class Work\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lata,40.1629,-8.3327,55.81,99,100,3.00,overcast clouds
1,1,Guerrero Negro,27.9769,-114.0611,64.60,56,100,6.89,overcast clouds
2,2,Rikitea,-23.1203,-134.9692,78.01,80,100,19.51,overcast clouds
3,3,Westport,41.1415,-73.3579,39.99,95,100,1.99,overcast clouds
4,4,Mataura,-46.1927,168.8643,69.48,45,42,1.90,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip?  70
What is the maximum temperature you would like for your trip?  90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,-23.1203,-134.9692,78.01,80,100,19.51,overcast clouds
7,7,Saint-Philippe,-21.3585,55.7679,76.59,85,100,11.97,light rain
8,8,Port Alfred,-33.5906,26.8910,70.75,91,100,20.22,overcast clouds
9,9,Bandarbeyla,9.4942,50.8122,76.71,61,88,18.81,overcast clouds
14,14,New Norfolk,-42.7826,147.0587,77.76,54,95,4.03,overcast clouds
17,17,Cabo San Lucas,22.8909,-109.9124,73.51,82,100,3.00,overcast clouds
18,18,Sao Filipe,14.8961,-24.4956,73.13,67,100,4.79,overcast clouds
21,21,Atuona,-9.8000,-139.0333,78.17,78,84,18.72,broken clouds
26,26,Hobart,-42.8794,147.3294,78.12,77,75,14.97,broken clouds
29,29,Busselton,-33.6500,115.3333,70.05,41,100,9.62,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                239
City                   239
Lat                    239
Lng                    239
Max Temp               239
Humidity               239
Cloudiness             239
Wind Speed             239
Current Description    239
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,-23.1203,-134.9692,78.01,80,100,19.51,overcast clouds
7,7,Saint-Philippe,-21.3585,55.7679,76.59,85,100,11.97,light rain
8,8,Port Alfred,-33.5906,26.8910,70.75,91,100,20.22,overcast clouds
9,9,Bandarbeyla,9.4942,50.8122,76.71,61,88,18.81,overcast clouds
14,14,New Norfolk,-42.7826,147.0587,77.76,54,95,4.03,overcast clouds
17,17,Cabo San Lucas,22.8909,-109.9124,73.51,82,100,3.00,overcast clouds
18,18,Sao Filipe,14.8961,-24.4956,73.13,67,100,4.79,overcast clouds
21,21,Atuona,-9.8000,-139.0333,78.17,78,84,18.72,broken clouds
26,26,Hobart,-42.8794,147.3294,78.12,77,75,14.97,broken clouds
29,29,Busselton,-33.6500,115.3333,70.05,41,100,9.62,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,78.01,overcast clouds,-23.1203,-134.9692,
7,Saint-Philippe,76.59,light rain,-21.3585,55.7679,
8,Port Alfred,70.75,overcast clouds,-33.5906,26.8910,
9,Bandarbeyla,76.71,overcast clouds,9.4942,50.8122,
14,New Norfolk,77.76,overcast clouds,-42.7826,147.0587,
17,Cabo San Lucas,73.51,overcast clouds,22.8909,-109.9124,
18,Sao Filipe,73.13,overcast clouds,14.8961,-24.4956,
21,Atuona,78.17,broken clouds,-9.8000,-139.0333,
26,Hobart,78.12,broken clouds,-42.8794,147.3294,
29,Busselton,70.05,overcast clouds,-33.6500,115.3333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,78.01,overcast clouds,-23.1203,-134.9692,People ThankYou
7,Saint-Philippe,76.59,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
8,Port Alfred,70.75,overcast clouds,-33.5906,26.8910,The Halyards Hotel
9,Bandarbeyla,76.71,overcast clouds,9.4942,50.8122,JABIR HOTEL
14,New Norfolk,77.76,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
...,...,...,...,...,...,...
669,Maridi,76.46,overcast clouds,4.9158,29.4742,
672,Santa Teresa,79.72,few clouds,10.2341,-66.6631,Funeraria Lira
673,Padang,80.65,moderate rain,-0.9492,100.3543,Mercure Padang
674,Boa Vista,78.78,clear sky,2.8197,-60.6733,Aipana Plaza Hotel


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,78.01,overcast clouds,-23.1203,-134.9692,People ThankYou
7,Saint-Philippe,76.59,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
8,Port Alfred,70.75,overcast clouds,-33.5906,26.8910,The Halyards Hotel
9,Bandarbeyla,76.71,overcast clouds,9.4942,50.8122,JABIR HOTEL
14,New Norfolk,77.76,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
17,Cabo San Lucas,73.51,overcast clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
18,Sao Filipe,73.13,overcast clouds,14.8961,-24.4956,Hotel Xaguate
21,Atuona,78.17,broken clouds,-9.8000,-139.0333,Villa Enata
26,Hobart,78.12,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
29,Busselton,70.05,overcast clouds,-33.6500,115.3333,Observatory Guest House


In [19]:
# 8a. Create the output File (CSV)
output_data_file = r"C:\Users\rverd\OneDrive\Bureaublad\Class Work\World_Weather_Analysis\Vacation_Search\WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
